# 3일차 4교시 - Spark Partitioning
>  스파크에서 성능을 개선하는 다양한 방법 가운데 파티셔닝과 버킷팅이 있습니다. 이 두가지 기법은 데이터를 저장하는 방법이며, 자주 혹은 함께 읽어지는 데이터 혹은 그룹을 저장 시에 함께 저장해둔다는 개념입니다. 데이터베이스와는 다르게 인덱스를 여러개 가지기 힘든 파일저장 구조의 한계를 극복하기 위해 가장 많이 사용하는 키 혹은 그룹의 정보를 파티셔닝이라는 방식으로 디렉토리를 구분하여 저장할 수 있습니다.

### 목차
* [1. '파티셔닝' 이란?](#1.-'파티셔닝'-이란?)
* [2. '파티셔닝' 잘하는 법?](#2.-'파티셔닝'-잘하는-법?)
* [3. 파티션 다루기](#3.-파티션-다루기)
* [4. 파티션의 특징](#4.-파티션의-특징)

### 참고자료
  * https://techmagie.wordpress.com/2015/12/19/understanding-spark-partitioning/

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

22/09/04 08:48:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 1. '파티셔닝' 이란?
> 분산환경에서 큰 데이터를 특정 노드에 모두 저장할 수 없기 때문에 여러 노드에 분산하여 저장하는데 이를 파티션이라고 하며, 스파크에서는 자동으로 RDD 를 파티셔닝 하고 분산 저장합니다. 스파크 내에서 파티션은 데이터 저장의 단위이자 병렬수행의 기본단위입니다.

#### 1.1 파티셔닝의 특징
* 하나의 익스큐터 당 하나의 파티션을 할당하므로 파티션 수가 전체 성능에 큰 영향을 미침 (ex_ sc.textFile("files", 5))
* 너무 파티션 수가 작은 경우
  * 병렬 수행의 장점을 누리기 어렵습니다
  * 데이터 편중에 따른 리소스 활용이 떨어질 수 있습니다
* 너무 파티션 수가 많은 경우
  * 실제 수행에 필요한 시간보다 작업 수행에 걸리는 시간이 더 오래 걸릴 수 있습니다
* 파티션 수와 성능은 항상 트레이드 오프 관계에 있으므로 밸런싱이 중요합니다
  * Average : 클러스터의 크기에 따라 다르지만 100 ~ 10K 수준의 파티션 수가 적절합니다
  * Lower Bound : 어플리케이션에 할당된 대상 클러스터 전체의 코어 수의 2배 정도입니다
  * Upper Bound : 반대로 타스크 수행에 100ms 가 안 걸린다면 데이터가 너무 작아 스케줄링에 더 많은 리소스가 소모되고 있습니다

#### 1.2. 파티셔닝의 종류
* None Partitioning → 파티셔닝이 데이터의 특성에 기반하지는 않지만, 모든 노드에 고르게 랜덤하게 분포됨을 말합니다
  * 데이터 Reading, Parallelize 등의 경우에는 None 
* Hash Partitioning → Object.hashCode 값을 기준으로 균등하게 노드에 배포하며 hashCode % numPartitions 로 결정합니다
  * reduceBy, groupBy, join 등과 같은 shuffle 이 발생하는 경우는 Hash
* Range Partitioning → RDD 의 키 가운데 순서가 의미있는 경우 가까운 범위의 값들을 정렬된 키를 기준으로 뭉쳐둘 수 있습니다
  * sortBy 와 같은 경우는 Range 입니다.

#### 1.3. 파티션은 언제 생성되는가?
* dataframe.partitionBy 과 같이 명시적으로 파티셔닝을 수행하거나
* shuffle 이 발생하는 연산(join, groupBy, reduceBy, foldBy, sort, partitionBy 연산) 수행 시

#### 1.4. 파티션 구성요소 확인
* glom() → 하나의 파티션에 존재하는 모든 요소 정보들을 모아 하나의 RDD 로 반환하는 API
* partitioner → RDD 로부터 partitioner 정보를 조사합니다
* 다양한 집계 및 조인 함수 수행 시에 셔플링이 발생하며 이 때에 파티셔닝 설계가 중요합니다

In [2]:
sc = spark.sparkContext
data = [1, 2, 3, 4, 5]

rdd = sc.parallelize(data)
print("Number of partitions: {}".format(rdd.getNumPartitions()))
print("Partitioner: {}".format(rdd.partitioner))
print("Partitions structure: {}".format(rdd.glom().collect()))

Number of partitions: 6
Partitioner: None


Partitions structure: [[], [1], [2], [3], [4], [5]]


<br>

### 2. '파티셔닝' 잘 하는 법
> 셔플이 발생하는 다양한 변환작업 중에 skew 가 발생한 특정 노드에 셔플 블락이 2GB 이상 집중되는 경우, 자바의 바이트 버퍼 제한으로 실행에 실패합니다

* 내가 사용하는 데이터소스의 크기, 타입과 데이터의 분포를 파악
  * 데이터소스의 split 가능여부, 편중되어 저장되어 있는지 혹은 파티셔닝이 균질하게 잘 저장되도록 전처리가 필요할 수도 있습니다
* 데이터 소스에 대해 어떠한 연산자를 사용하는 것이 적절한 지 파악
  * 어떤 경우에 어떤 집계 연산을 사용하면 좋은지 팥단이 필요함 - reduceByKey, aggregateByKey
* 임의의 데이터 소스 혹은 셔플 키에 대한 파티셔닝은 솔팅 기법을 적용하는 것을 검토
  * 편중이 심한 파티션 그룹에 대해 상대적으로 큰 파티션 조인 파트에 뻥튀기가 되도록 explode 통한 salting key 부여
  * 나머지 파티션 조인 파트에는 동일한 키 그룹에 대해 salting key 부여 하여 해당 키 가지수 만큼 병렬성 부여

<br>

### 3. 파티션 다루기

In [3]:
flight = spark.read.parquet(f"{work_data}/flight-data/parquet/2010-summary.parquet")
flight.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [4]:
flight.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
|    United States|          Singapore|   25|
|    United States|            Grenada|   54|
|       Costa Rica|      United States|  477|
|          Senegal|      United States|   29|
|    United States|   Marshall Islands|   44|
+-----------------+-------------------+-----+
only showing top 10 rows



In [4]:
flight.write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME").parquet("target/troubleshoot4")

In [5]:
united_states = spark.read.parquet("target/troubleshoot4").where("DEST_COUNTRY_NAME='United States'")
united_states.printSchema()
united_states.show(10)

root
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)
 |-- DEST_COUNTRY_NAME: string (nullable = true)

+--------------------+-----+-----------------+
| ORIGIN_COUNTRY_NAME|count|DEST_COUNTRY_NAME|
+--------------------+-----+-----------------+
|             Romania|    1|    United States|
|             Ireland|  264|    United States|
|               India|   69|    United States|
|           Singapore|   25|    United States|
|             Grenada|   54|    United States|
|    Marshall Islands|   44|    United States|
|        Sint Maarten|   53|    United States|
|         Afghanistan|    2|    United States|
|              Russia|  156|    United States|
|Federated States ...|   48|    United States|
+--------------------+-----+-----------------+
only showing top 10 rows



In [6]:
flight.write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").parquet("target/troubleshoot4")

In [7]:
us_sk = spark.read.parquet("target/troubleshoot4").where("DEST_COUNTRY_NAME='United States' and ORIGIN_COUNTRY_NAME='South Korea'")
us_sk.printSchema()
us_sk.show(10)

root
 |-- count: long (nullable = true)
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)

+-----+-----------------+-------------------+
|count|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----+-----------------+-------------------+
|  621|    United States|        South Korea|
+-----+-----------------+-------------------+



<br>

### 4. 파티셔닝 주의할 점
> 파티션 구성 시에 데이터의 크기에 따른 파티션 선정이 중요한데 너무 작은 데이터 파티션에서 너무 다양한 파티션 값을 가진 경우 오히려 성능이 떨어지는 경우도 발생할 수 있습니다.

* 파티션의 수가 너무 많은 경우 성능이 떨어지는 예제
* 파티션의 수에 따라 파일 I/O가 달라지는 예제

> 로컬 환경에서 수백MB 데이터 예제 수행 시에 정상적으로 동작하지 않기 때문에 성능이 더 좋아지는 예제를 포함하지 못한 점 양해 부탁 드립니다.

In [9]:
normal = spark.read.option("inferSchema", True).json(f"{work_data}/flight-data/json")

In [10]:
normal.write.mode("overwrite").parquet("target/part_v1")

In [11]:
normal.write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME").parquet("target/part_v2")

In [19]:
%%time
normal = spark.read.parquet("target/part_v1")
normal_expr = normal.where(expr("DEST_COUNTRY_NAME = 'United States'")).groupBy("ORIGIN_COUNTRY_NAME").count()
normal_expr.explain()
normal_expr.show(1, truncate=False)

== Physical Plan ==
*(2) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#368], functions=[count(1)])
+- Exchange hashpartitioning(ORIGIN_COUNTRY_NAME#368, 5), ENSURE_REQUIREMENTS, [id=#432]
   +- *(1) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#368], functions=[partial_count(1)])
      +- *(1) Project [ORIGIN_COUNTRY_NAME#368]
         +- *(1) Filter (isnotnull(DEST_COUNTRY_NAME#367) AND (DEST_COUNTRY_NAME#367 = United States))
            +- *(1) ColumnarToRow
               +- FileScan parquet [DEST_COUNTRY_NAME#367,ORIGIN_COUNTRY_NAME#368] Batched: true, DataFilters: [isnotnull(DEST_COUNTRY_NAME#367), (DEST_COUNTRY_NAME#367 = United States)], Format: Parquet, Location: InMemoryFileIndex[file:/home/jovyan/work/lgde-spark-troubleshoot/target/part_v1], PartitionFilters: [], PushedFilters: [IsNotNull(DEST_COUNTRY_NAME), EqualTo(DEST_COUNTRY_NAME,United States)], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string>


+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------

In [20]:
%%time
partitioned = spark.read.parquet("target/part_v2")
partitioned_expr = partitioned.where(expr("DEST_COUNTRY_NAME = 'United States'")).groupBy("ORIGIN_COUNTRY_NAME").count()
partitioned_expr.explain()
partitioned_expr.show(1, truncate=False)

== Physical Plan ==
*(2) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#391], functions=[count(1)])
+- Exchange hashpartitioning(ORIGIN_COUNTRY_NAME#391, 5), ENSURE_REQUIREMENTS, [id=#498]
   +- *(1) HashAggregate(keys=[ORIGIN_COUNTRY_NAME#391], functions=[partial_count(1)])
      +- *(1) Project [ORIGIN_COUNTRY_NAME#391]
         +- *(1) ColumnarToRow
            +- FileScan parquet [ORIGIN_COUNTRY_NAME#391,DEST_COUNTRY_NAME#393] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex[file:/home/jovyan/work/lgde-spark-troubleshoot/target/part_v2], PartitionFilters: [isnotnull(DEST_COUNTRY_NAME#393), (DEST_COUNTRY_NAME#393 = United States)], PushedFilters: [], ReadSchema: struct<ORIGIN_COUNTRY_NAME:string>


+-------------------+-----+
|ORIGIN_COUNTRY_NAME|count|
+-------------------+-----+
|Ireland            |6    |
+-------------------+-----+
only showing top 1 row

CPU times: user 5.92 ms, sys: 4.35 ms, total: 10.3 ms
Wall time: 1.91 s


In [14]:
activity = spark.read.option("inferSchema", True).json(f"{work_data}/activity-data")
activity.printSchema()
activity.show(1, truncate=False)

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)

+-------------+-------------------+--------+-----+------+----+-----+------------+-----------+-----------+
|Arrival_Time |Creation_Time      |Device  |Index|Model |User|gt   |x           |y          |z          |
+-------------+-------------------+--------+-----+------+----+-----+------------+-----------+-----------+
|1424686734964|1424688581018556031|nexus4_2|1    |nexus4|g   |stand|-0.001449585|0.035491943|0.027999878|
+-------------+-------------------+--------+-----+------+----+-----+------------+-----------+-----------+
only showing top 1 row



In [15]:
activity.write.mode("overwrite").parquet("target/normal")

In [16]:
activity.write.partitionBy("User").mode("overwrite").parquet("target/partition")

In [17]:
%%time
spark.read.parquet("target/normal").where("User = 'e'").groupBy("gt").count()

CPU times: user 2 ms, sys: 2.68 ms, total: 4.69 ms
Wall time: 120 ms


gt,count
sit,8364
stand,8058
stairsdown,5995
walk,9480
null,10476
stairsup,6557
bike,8688


In [18]:
%%time
spark.read.parquet("target/partition").where("User = 'e'").groupBy("gt").count()

CPU times: user 2.32 ms, sys: 1.35 ms, total: 3.68 ms
Wall time: 266 ms


gt,count
sit,8364
stand,8058
stairsdown,5995
walk,9480
null,10476
stairsup,6557
bike,8688


In [43]:
%%time
activity.unpersist()
activity.where("User = 'e'").groupBy("gt").count()

CPU times: user 2.4 ms, sys: 1.85 ms, total: 4.25 ms
Wall time: 13.2 ms


gt,count
sit,4182
stand,4029
stairsdown,2997
walk,4740
null,5238
stairsup,3279
bike,4344


<br>

### 5. 파티션의 특징

* Number of Tasks on per stage basis = Number of partitions
  * 특정 스테이지의 타스크 수는 파티션 수와 같습니다
  * 즉, 
* 같은 파티션에 존재하는 튜플들은 반드시 같은 장비에 존재한다
* 하나의 파티션에는 하나의 타스크만 할당되며, 워커는 한 번에 하나의 타스크만 수행합니다
* 스파크 셔플 블럭들의 크기는 최대 2GB를 넘지 못합니다
  * 자바 바이트버퍼 추상화 객체의 MAX_SIZE 가 2GB 이기 때문입니다